In [5]:
import numpy as np
import pandas as pd
import warnings
import math
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.linear_model import LinearRegression

In [47]:
x1 = [1,6,8,3,7,9,10,15,20]
x2 = [4,5,6,7,8,1,12,18,27]
y = [9,15,18,14,18,13,24,32,41]


data = pd.DataFrame({'x1': x1,'x2':x2, 'y':y})




In [48]:
df = pd.DataFrame(data)

In [49]:
df.head()

,x1,x2,y
0,1,4,9
1,6,5,15
2,8,6,18
3,3,7,14
4,7,8,18


In [50]:
# Separamos en train y test

train = df.iloc[:7]
test = df.iloc[7:]

train_X = train.drop('y', axis=1)
train_y = train.y

test_X = test.drop('y', axis=1)
test_y = test.y

In [51]:
test_X

,x1,x2
7,15,18
8,20,27


In [52]:
#  verificacmos si la regresión lineal puede predecir correctamente

m1 = LinearRegression()
fit1 = m1.fit(train_X, train_y)
preds = fit1.predict(test_X)
print(f"\n{preds}")
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds))}\n")


[34.16539092 46.68698742]
RMSE: 4.302949210938829



In [55]:
# Las predicciones no son malas, pero hagamos algunos cálculos sobre las características de entrada para mejorar
# ¿Qué pasa si sacamos la raíz cuadrada de X2 y multiplicamos por 2?

train_X.x2 = 4 * np.sqrt(train_X.x2)

test_X.x2 = 4 * np.sqrt(test_X.x2)

print(test_X)

m2 = LinearRegression()

fit2 = m2.fit(train_X, train_y)

preds = fit2.predict(test_X)

print(f"\nPred:{preds}")

print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds))}\n")

   x1         x2
7  15  16.970563
8  20  20.784610

Pred:[32.014806   40.80754258]
RMSE: 0.13649006477128428



In [56]:
# Restauración

train = df.iloc[:7]
test = df.iloc[7:]

train_X = train.drop('y', axis=1)
train_y = train.y

test_X = test.drop('y', axis=1)
test_y = test.y

In [63]:
# Creación de clase transformadora

class ExperimentalTransformer(BaseEstimator, TransformerMixin):
  def __init__(self):
    print('\n>>>>>>>init() called.\n')

  def fit(self, X, y = None):
    print('\n>>>>>>>fit() called.\n')
    return self

  def transform(self, X, y = None):
    print('\n>>>>>>>transform() called.\n')
    X_ = X.copy() 
    X_.x2 = 4 * np.sqrt(X_.x2)
    return X_

In [64]:
# Prueba pipeline sin transformación


print("create pipeline 1")
pipe1 = Pipeline(steps=[
                       ('linear_model', LinearRegression())
])

print("fit pipeline 1")
pipe1.fit(train_X, train_y)

print("predict via pipeline 1")
preds1 = pipe1.predict(test_X)

print(f"\n{preds1}")  # should be [13.72113586 16.93334467]
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds1))}\n")

create pipeline 1
fit pipeline 1
predict via pipeline 1

[34.16539092 46.68698742]
RMSE: 4.302949210938829



In [60]:
# Prueba pipeline con transformación
print("create pipeline 2")
pipe2 = Pipeline(steps=[
                       ('experimental_trans', ExperimentalTransformer()),    # esto dispara una llamda a __init__
                       ('linear_model', LinearRegression())
])


print("fit pipeline 2")
pipe2.fit(train_X, train_y)

print("predict via pipeline 2")
preds2 = pipe2.predict(test_X)

print(f"\n{preds2}")  # should be [14. 17.]
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds2))}\n")



create pipeline 2

>>>>>>>init() called.

fit pipeline 2

>>>>>>>fit() called.


>>>>>>>transform() called.

predict via pipeline 2

>>>>>>>transform() called.


[32.014806   40.80754258]
RMSE: 0.13649006477128428

